In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
from tqdm import tqdm_notebook as tqdm
import random
from sklearn.dummy import DummyClassifier
from scipy import stats

In [3]:
path_to_data = "../data/train_images.npy"
path_to_labels = "../data/train_labels.csv"
path_to_test = "../data/test_images.npy"

In [8]:
data = np.load(path_to_data, encoding = 'bytes')
labels_df = pd.read_csv(path_to_labels)
labels_df.Category = pd.Categorical(labels_df.Category)
y = labels_df.Category.cat.codes.values

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_images.npy'

In [5]:
def plot_images(list_of_images, max_col = 4):
    n = len(list_of_images)
    if n == 1:
        plt.imshow(list_of_images[0], cmap = 'gray_r'); plt.axis('off'); plt.show()
    else:
        # get number of columns and rows required
        r, c = 1, n
        if n > max_col:
            c = max_col
            r = int(math.ceil(n/max_col))
    
        fig = plt.figure(figsize=(17, max_col * r))
        for i, (img,name) in enumerate(list_of_images):
            ax = fig.add_subplot(r, c, (i+1))
            ax.set_title(str(name))
            ax.axis('off')
            ax.imshow(img, cmap = 'gray_r')

In [6]:
X = list(data[:, 1])

NameError: name 'data' is not defined

In [7]:
x_tr, x_val, y_tr, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

NameError: name 'X' is not defined

In [225]:
x_tr = np.array([x.reshape(100, 100) for x in x_tr])
x_val = np.array([x.reshape(100, 100) for x in x_val])

In [233]:
x_tr.shape

(6700, 100, 100)

In [240]:
def performance_random_clf(x_tr, y_tr, x_test, y_test, r = 1234):
    clf = DummyClassifier(strategy = 'uniform', random_state = r)
    clf.fit(x_tr, y_tr)
    preds = clf.predict(x_test)
    metric = metrics.classification_report(y_test, preds)
    return metric
    
def performance_majority_classifier(x_tr, y_tr, x_test, y_test, r = 1234):
    most_common_val = stats.mode(y_tr).mode[0]
    preds = np.full((y_test.shape), most_common_val)
    metric = metrics.classification_report(y_test, preds)
    return metric

In [241]:
print(performance_random_clf(x_tr, y_tr, x_val, y_val))
print(performance_majority_classifier(x_tr, y_tr, x_val, y_val))

              precision    recall  f1-score   support

           0       0.03      0.03      0.03       110
           1       0.04      0.04      0.04       108
           2       0.06      0.04      0.04       168
           3       0.08      0.04      0.05       204
           4       0.07      0.05      0.06       142
           5       0.04      0.03      0.03       139
           6       0.00      0.00      0.00        56
           7       0.03      0.02      0.02       158
           8       0.04      0.04      0.04        95
           9       0.02      0.03      0.02        68
          10       0.03      0.05      0.04        62
          11       0.02      0.03      0.02        63
          12       0.04      0.08      0.05        51
          13       0.05      0.06      0.05        89
          14       0.04      0.04      0.04       102
          15       0.02      0.03      0.03        93
          16       0.05      0.04      0.05       142
          17       0.05    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [229]:
lSVM_cs = [1,10000]
n_fine_tune = 50

In [251]:
from sklearn import svm
def linear_SVM(x_tr, y_tr, x_val, y_val):
    f1_scores = []
    model = svm.SVC()
    model.fit(x_tr, y_tr)
    preds = model.predict(x_val)
    metric = metrics.classification_report(y_val, preds)

    return model, metric

In [252]:
a, b = linear_SVM(x_tr, y_tr, x_val, y_val)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [253]:
print(a)
print(b)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       110
           1       0.00      0.00      0.00       108
           2       0.00      0.00      0.00       168
           3       0.06      1.00      0.12       204
           4       0.00      0.00      0.00       142
           5       0.00      0.00      0.00       139
           6       0.00      0.00      0.00        56
           7       0.00      0.00      0.00       158
           8       0.00      0.00      0.00        95
           9       0.00      0.00      0.00        68
          10       0.00      0.00      0.00        62
          11       0.00      0.00      0.00        63
          12       0.00      0.00      0.00        51
       

In [1]:
from sklearn import svm
def fine_tune_SVM(x_tr, y_tr, x_val, y_val,
                         number_of_model, cs):
    all_c = [0.001, 0.01, 0.1, 0.5, 1, 2, 10, 50, 100, 1000]
    all_gamma = [0.001, 0.01, 0.1, 0.5, 1, 2, 10, 50, 100, 1000]
    
    f1_scores = []
    print(all_c)
    best_model = None
    for c, g in tqdm(zip(all_c, all_gamma)):
        model = svm.SVC(C=c, gamma = g)
        model.fit(x_tr, y_tr)
        preds = model.predict(x_val)
        cur_f1 = metrics.f1_score(y_val, preds, average = 'micro')
        f1_scores.append(cur_f1)
        if cur_f1 == max(f1_scores):
            best_model = model
    print("The values of Cs considered are:", all_c)
    print("The best f1 score is:", max(f1_scores))
    print("The c is set to:", best_model.get_params()['C'])
    return best_model

In [219]:
svm = fine_tune_linear_SVM(x_tr, y_tr, x_val, y_val, n_fine_tune, lSVM_cs)

[0.001, 0.01, 0.1, 0.5, 1, 2, 10, 50, 100, 1000]


HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

KeyboardInterrupt: 

In [ ]:
svm